# Exploratory analysis

In [1]:
import intake
import pandas as pd

catalog = intake.open_catalog("../catalog.yml")
# If we had a `catalog` folder with lots of YAML files, use the * to designate everything with a .yml:
# catalog = intake.open_catalog("../catalogs/*.yml")

## JHU cases data

A lot of this is directly imported from JHU. There are some added columns, starting from `state_cases`, all the way to `new_state_deaths`. Those are columns added when the data was imported and saved.

Let's poke around and see if we want to generate any new columns.

In [2]:
jhu = catalog.jhu_cases.read()
jhu.head()

,county,state,fips,date,Lat,Lon,cases,deaths,incident_rate,people_tested,state_cases,state_deaths,new_cases,new_deaths,new_state_cases,new_state_deaths
62,Autauga,Alabama,01001,2020-03-24 07:00:00+00:00,32.539527,-86.644082,1,0,1.789901,NaN,298,0,1,0,74,0
63,Autauga,Alabama,01001,2020-03-25 07:00:00+00:00,32.539527,-86.644082,5,0,8.949507,NaN,472,1,4,0,174,1
64,Autauga,Alabama,01001,2020-03-26 07:00:00+00:00,32.539527,-86.644082,6,0,10.739408,NaN,574,1,1,0,102,0
65,Autauga,Alabama,01001,2020-03-27 07:00:00+00:00,32.539527,-86.644082,6,0,10.739408,NaN,684,4,0,0,110,3
66,Autauga,Alabama,01001,2020-03-28 07:00:00+00:00,32.539527,-86.644082,6,0,10.739408,NaN,801,4,0,0,117,0


In [ ]:
"""
Alternatively, can read it in using pandas.

jhu = pd.read_parquet("../data/us-county-time-series.parquet")
"""

In [3]:
# Data types
# strings are objects
# numeric can be floats (with decimals) or integers
# datetime
jhu.dtypes

county                           object
state                            object
fips                             object
date                datetime64[ns, UTC]
Lat                             float64
Lon                             float64
cases                             int64
deaths                            int64
incident_rate                   float64
people_tested                   float64
state_cases                       int64
state_deaths                      int64
new_cases                         int64
new_deaths                        int64
new_state_cases                   int64
new_state_deaths                  int64
dtype: object

In [4]:
# Subset data
la = jhu[jhu.county=="Los Angeles"] 
la.head()

,county,state,fips,date,Lat,Lon,cases,deaths,incident_rate,people_tested,state_cases,state_deaths,new_cases,new_deaths,new_state_cases,new_state_deaths
71781,Los Angeles,California,06037,2020-01-26 08:00:00+00:00,34.308284,-118.228241,1,0,0.009961,NaN,2,0,1,0,2,0
71782,Los Angeles,California,06037,2020-01-27 08:00:00+00:00,34.308284,-118.228241,1,0,0.009961,NaN,2,0,0,0,0,0
71783,Los Angeles,California,06037,2020-01-28 08:00:00+00:00,34.308284,-118.228241,1,0,0.009961,NaN,2,0,0,0,0,0
71784,Los Angeles,California,06037,2020-01-29 08:00:00+00:00,34.308284,-118.228241,1,0,0.009961,NaN,2,0,0,0,0,0
71785,Los Angeles,California,06037,2020-01-30 08:00:00+00:00,34.308284,-118.228241,1,0,0.009961,NaN,2,0,0,0,0,0


In [5]:
# We might need this
import useful_dict

useful_dict.us_state_abbrev

{'Alabama': 'AL',
 'Alaska': 'AK',
 'American Samoa': 'AS',
 'Arizona': 'AZ',
 'Arkansas': 'AR',
 'California': 'CA',
 'Colorado': 'CO',
 'Connecticut': 'CT',
 'Delaware': 'DE',
 'District of Columbia': 'DC',
 'Florida': 'FL',
 'Georgia': 'GA',
 'Guam': 'GU',
 'Hawaii': 'HI',
 'Idaho': 'ID',
 'Illinois': 'IL',
 'Indiana': 'IN',
 'Iowa': 'IA',
 'Kansas': 'KS',
 'Kentucky': 'KY',
 'Louisiana': 'LA',
 'Maine': 'ME',
 'Maryland': 'MD',
 'Massachusetts': 'MA',
 'Michigan': 'MI',
 'Minnesota': 'MN',
 'Mississippi': 'MS',
 'Missouri': 'MO',
 'Montana': 'MT',
 'Nebraska': 'NE',
 'Nevada': 'NV',
 'New Hampshire': 'NH',
 'New Jersey': 'NJ',
 'New Mexico': 'NM',
 'New York': 'NY',
 'North Carolina': 'NC',
 'North Dakota': 'ND',
 'Northern Mariana Islands': 'MP',
 'Ohio': 'OH',
 'Oklahoma': 'OK',
 'Oregon': 'OR',
 'Pennsylvania': 'PA',
 'Puerto Rico': 'PR',
 'Rhode Island': 'RI',
 'South Carolina': 'SC',
 'South Dakota': 'SD',
 'Tennessee': 'TN',
 'Texas': 'TX',
 'Utah': 'UT',
 'Vermont': 'VT',
 '

In [6]:
# Make new columns

# Can use .assign() to create multiple new columns at once
jhu = jhu.assign(
    # Make the datetime into a string, lose the timestamp
    date = pd.to_datetime(jhu.date).dt.date,
    county_state = jhu.county + ", " + jhu.state,
)

# Or, we can do it one at a time
# Make date2, which can be a datetime column, so it recognizes various forms of dates
jhu['date2'] = pd.to_datetime(jhu.date)    
jhu['state_abbrev'] = jhu.state.map(useful_dict.us_state_abbrev)

In [7]:
jhu[(jhu.date2 == "2020-12-23") & (jhu.county=="Riverside")]

,county,state,fips,date,Lat,Lon,cases,deaths,incident_rate,people_tested,state_cases,state_deaths,new_cases,new_deaths,new_state_cases,new_state_deaths,county_state,date2,state_abbrev
77167,Riverside,California,06065,2020-12-23,33.74315,-115.993358,151713,1789,6140.869265,NaN,1964945,23308,0,0,6437,69,"Riverside, California",2020-12-23,CA


In [8]:
jhu[(jhu.date2 == "12/23/20") & (jhu.county=="Riverside")]

,county,state,fips,date,Lat,Lon,cases,deaths,incident_rate,people_tested,state_cases,state_deaths,new_cases,new_deaths,new_state_cases,new_state_deaths,county_state,date2,state_abbrev
77167,Riverside,California,06065,2020-12-23,33.74315,-115.993358,151713,1789,6140.869265,NaN,1964945,23308,0,0,6437,69,"Riverside, California",2020-12-23,CA


In [9]:
jhu[(jhu.date2 == "12-23-20") & (jhu.county=="Riverside")]

,county,state,fips,date,Lat,Lon,cases,deaths,incident_rate,people_tested,state_cases,state_deaths,new_cases,new_deaths,new_state_cases,new_state_deaths,county_state,date2,state_abbrev
77167,Riverside,California,06065,2020-12-23,33.74315,-115.993358,151713,1789,6140.869265,NaN,1964945,23308,0,0,6437,69,"Riverside, California",2020-12-23,CA


In [10]:
# But `date` doesn't work like the datetime, which is why we use `date2`
jhu[(jhu.date == "12-23-20") & (jhu.county=="Riverside")].head()

,county,state,fips,date,Lat,Lon,cases,deaths,incident_rate,people_tested,state_cases,state_deaths,new_cases,new_deaths,new_state_cases,new_state_deaths,county_state,date2,state_abbrev


## Population crosswalk

In [11]:
crosswalk = catalog.msa_county_crosswalk.read()
crosswalk.head()

,cbsacode,cbsatitle,metro_micro,county,state,county_fips,fips_state_code,fips_county_code,county_pop,msa_pop
0,10100,"Aberdeen, SD",Micropolitan Statistical Area,Brown County,South Dakota,46013,46,13,38839.0,42668.0
1,10100,"Aberdeen, SD",Micropolitan Statistical Area,Edmunds County,South Dakota,46045,46,45,3829.0,42668.0
2,10140,"Aberdeen, WA",Micropolitan Statistical Area,Grays Harbor County,Washington,53027,53,27,75061.0,75061.0
3,10180,"Abilene, TX",Metropolitan Statistical Area,Callahan County,Texas,48059,48,59,13943.0,172060.0
4,10180,"Abilene, TX",Metropolitan Statistical Area,Jones County,Texas,48253,48,253,20083.0,172060.0


In [12]:
# Look at catalog.yml
# CSVs are bad at keeping data types, so we can force certain columns to be read in with certain data types
# Here, cbsacode and county_fips are forced to be read in as string
# fips is string in JHU data
crosswalk.dtypes

cbsacode             object
cbsatitle            object
metro_micro          object
county               object
state                object
county_fips          object
fips_state_code       int64
fips_county_code      int64
county_pop          float64
msa_pop             float64
dtype: object

In [13]:
# Let's rename a column
crosswalk = crosswalk.rename(columns = {"county_fips": "fips"})

In [14]:
# We can also string a bunch of commands together
crosswalk = (crosswalk.rename(columns = {"cbsacode": "cbsa_code"})
             [["fips", "county_pop", "cbsatitle"]]
            )

crosswalk.head()

,fips,county_pop,cbsatitle
0,46013,38839.0,"Aberdeen, SD"
1,46045,3829.0,"Aberdeen, SD"
2,53027,75061.0,"Aberdeen, WA"
3,48059,13943.0,"Abilene, TX"
4,48253,20083.0,"Abilene, TX"


## Merge cases with population

In [15]:
df = pd.merge(jhu, 
              # We'll subset the crosswalk df right here
              crosswalk[["fips", "county_pop"]], 
              on = "fips", 
              how = "inner", 
              validate = "m:1")

df.head()

,county,state,fips,date,Lat,Lon,cases,deaths,incident_rate,people_tested,state_cases,state_deaths,new_cases,new_deaths,new_state_cases,new_state_deaths,county_state,date2,state_abbrev,county_pop
0,Autauga,Alabama,01001,2020-03-24,32.539527,-86.644082,1,0,1.789901,NaN,298,0,1,0,74,0,"Autauga, Alabama",2020-03-24,AL,55869.0
1,Autauga,Alabama,01001,2020-03-25,32.539527,-86.644082,5,0,8.949507,NaN,472,1,4,0,174,1,"Autauga, Alabama",2020-03-25,AL,55869.0
2,Autauga,Alabama,01001,2020-03-26,32.539527,-86.644082,6,0,10.739408,NaN,574,1,1,0,102,0,"Autauga, Alabama",2020-03-26,AL,55869.0
3,Autauga,Alabama,01001,2020-03-27,32.539527,-86.644082,6,0,10.739408,NaN,684,4,0,0,110,3,"Autauga, Alabama",2020-03-27,AL,55869.0
4,Autauga,Alabama,01001,2020-03-28,32.539527,-86.644082,6,0,10.739408,NaN,801,4,0,0,117,0,"Autauga, Alabama",2020-03-28,AL,55869.0


## Use functions
Can use functions to roll-up several steps we want to do in one.
A function keeps your code clean, and allows you to reuse certain parts over and over without writing it again.

In [16]:
# Calculate a 7-day rolling average
# Takes as an argument, or input, a dataframe.
# When we invoke the function, the dataframe doesn't have to be called `df` necessarily.
# new_df = calculate_rolling_average(old_df)

def calculate_rolling_average(df):
    # Drop any NaNs or rolling average will choke
    df = df.dropna(subset = ["new_cases", "new_deaths"])
    
    # Derive new columns
    df = df.assign(
        cases_avg7=df.new_cases.rolling(window=7).mean(),
        deaths_avg7=df.new_deaths.rolling(window=7).mean(),
    )     
        
    return df

In [18]:
# Clean the JHU county data at once
def clean_jhu():
    # Import data
    df = catalog.jhu_cases.read()
    
    # Get some new columns
    df = df.assign(
        date=pd.to_datetime(df.date).dt.date,
        state_abbrev=df.state.map(useful_dict.us_state_abbrev),
    )
    
    # Define which columns to keep, drop the rest
    keep_cols = [
        "county",
        "state",
        "state_abbrev",
        "fips",
        "date",
        "Lat",
        "Lon",
        "cases",
        "deaths",
        "new_cases",
        "new_deaths",
    ]
    
    # We'll subset and only keep certain columns
    # Then sort our data according to county-state-fips-date, so everything is in the right order
    # before we calculate rolling averages
    # reset_index just re-orders our index so everything is labeled in the right sort order
    df1 = (df[keep_cols]
        .sort_values(["county", "state", "fips", "date"])
        .reset_index(drop=True)
    )
    
    # Bring in population crosswalk
    pop = catalog.msa_county_crosswalk.read()
    
    # Only need certain columns
    pop = (pop[["county_fips", "county_pop"]]
           .rename(columns = {"county_fips": "fips"})
          )
    
    # Merge cases with population
    df2 = pd.merge(df1, pop,
                  on = "fips", how = "inner", validate = "m:1"
    )
    
    # Call the function we just defined above
    # Can roll-up sub-functions into main functions as one way to keep things clean
    df3 = calculate_rolling_average(df2)
    
    return df3

In [24]:
cases_df = clean_jhu()
cases_df.head()

,county,state,state_abbrev,fips,date,Lat,Lon,cases,deaths,new_cases,new_deaths,county_pop,cases_avg7,deaths_avg7
0,Abbeville,South Carolina,SC,45001,2020-03-19,34.223334,-82.461707,1,0,1,0,24527.0,NaN,NaN
1,Abbeville,South Carolina,SC,45001,2020-03-20,34.223334,-82.461707,1,0,0,0,24527.0,NaN,NaN
2,Abbeville,South Carolina,SC,45001,2020-03-21,34.223334,-82.461707,1,0,0,0,24527.0,NaN,NaN
3,Abbeville,South Carolina,SC,45001,2020-03-22,34.223334,-82.461707,1,0,0,0,24527.0,NaN,NaN
4,Abbeville,South Carolina,SC,45001,2020-03-23,34.223334,-82.461707,1,0,0,0,24527.0,NaN,NaN


In [26]:
ca = cases_df[cases_df.state_abbrev=="CA"]
ca.head()

,county,state,state_abbrev,fips,date,Lat,Lon,cases,deaths,new_cases,new_deaths,county_pop,cases_avg7,deaths_avg7
5150,Alameda,California,CA,06001,2020-03-03,37.646294,-121.892927,1,0,1,0,1671329.0,76.857143,0.285714
5151,Alameda,California,CA,06001,2020-03-04,37.646294,-121.892927,1,0,0,0,1671329.0,53.571429,0.285714
5152,Alameda,California,CA,06001,2020-03-05,37.646294,-121.892927,1,0,0,0,1671329.0,42.428571,0.285714
5153,Alameda,California,CA,06001,2020-03-06,37.646294,-121.892927,1,0,0,0,1671329.0,24.857143,0.285714
5154,Alameda,California,CA,06001,2020-03-07,37.646294,-121.892927,2,0,1,0,1671329.0,15.142857,0.000000
